# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [ ]:
import sqlite3    # Python interface for SQLite databases

In [ ]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/SD202-Databases/wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
AllWines:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
Production:
  0: NV(NUM)
  1: CRU(TEXT)
  2: QTE(NUM)
  3: NP(NUM)
  4: REGION(TEXT)
Sales:
  0: NV(NUM)
  1: LIEU(TEXT)
  2: QTE(NUM)
  3: NB(NUM)
  4: REGION(TEXT)
WineProd:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WineSales:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WineTable:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [ ]:
%load_ext sql
%sql sqlite:////content/drive/MyDrive/SD202-Databases/wine.db

'Connected: @/content/drive/MyDrive/SD202-Databases/wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [ ]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [ ]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [ ]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.
Done.


[]

In [ ]:
print("\nContent of the database")
printSchema(conn)


Content of the database
AllWines:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
Production:
  0: NV(NUM)
  1: CRU(TEXT)
  2: QTE(NUM)
  3: NP(NUM)
  4: REGION(TEXT)
Sales:
  0: NV(NUM)
  1: LIEU(TEXT)
  2: QTE(NUM)
  3: NB(NUM)
  4: REGION(TEXT)
WineProd:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WineSales:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WineTable:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
dummy:
  0: DEGRE(NUM)


In [ ]:
%%sql
SELECT *
FROM dummy;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [ ]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [ ]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [ ]:
%%sql DROP TABLE IF EXISTS WineProd;

-- Creating Wine table
CREATE TABLE WineProd AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;

SELECT *
FROM WineProd

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [ ]:
%%sql DROP TABLE IF EXISTS Production;

-- Creating Production table
CREATE TABLE Production AS
SELECT DISTINCT NV, CRU, QTE, NP, REGION
FROM MASTER1
WHERE NV IS NOT NULL;

SELECT *
FROM Production;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.
Done.
Done.


NV,CRU,QTE,NP,REGION
1,Mercurey,300,1,Alsace
1,Mercurey,1,73,Jura
2,Julienas,100,5,Bourgogne
3,Savigny les Beaunes,400,1,Alsace
4,Mercurey,35,10,Bourgogne
5,Pommard,46,30,Bourgogne
6,Mercurey,300,42,Bourgogne
7,Grands Echezeaux,60,98,Rhone
8,Cotes de Beaune Villages,12,90,Rhone
9,Chapelle Chambertin,None,None,None


In [ ]:
%%sql DROP TABLE IF EXISTS Producer;

-- Creating Producer table
CREATE TABLE Producer AS
SELECT DISTINCT NP, NOM, PRENOM
FROM MASTER1
WHERE NP IS NOT NULL
ORDER BY NP;

SELECT *
FROM Producer;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.
Done.
Done.


NP,NOM,PRENOM
1,Bohn,Rene
2,Boxler,Albert
3,Six,Paul
4,Stentz,Fernand
5,Joudeat,Lucien
6,Marmagne,Bernard
7,Lasnier,Pierre
8,Lioger d'Harduy,Gabriel
9,Moniot,Gilbert
10,Fournier,Jean Claude


#### Exercise 1.4

Create the new tables from Master2:

In [ ]:
%%sql DROP TABLE IF EXISTS WineSales;

-- Creating Wine Sales table
CREATE TABLE WineSales AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL;

SELECT *
FROM WineSales;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [ ]:
%%sql DROP TABLE IF EXISTS Sales;

-- Creating  Sales table
CREATE TABLE Sales AS
SELECT DISTINCT NV, LIEU, QTE, NB, REGION
FROM MASTER2
WHERE NV IS NOT NULL;

SELECT *
FROM Sales;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.
Done.
Done.


NV,LIEU,QTE,NB,REGION
1,BORDEAUX,33,2,NOUVELLE-AQUITAINE
1,PARIS,1,44,ÎLE-DE-FRANCE
1,RENNES,1,45,BRETAGNE
2,LYON,2,48,AUVERGNE-RHÔNE-ALPES
3,NICE,6,7,PROVENCE-ALPES-CÔTE D'AZUR
3,MERCUREY,5,49,BOURGOGNE-FRANCHE-COMTÉ
4,TOULOUSE,12,8,OCCITANIE
4,PARIS,6,44,ÎLE-DE-FRANCE
4,SENS,4,50,BOURGOGNE-FRANCHE-COMTÉ
5,MERCUREY,50,2,BOURGOGNE-FRANCHE-COMTÉ


In [ ]:
%%sql

DELETE 
FROM Sales
WHERE LIEU IS NULL 
      AND	QTE IS NULL
      AND	NB IS NULL
      AND	REGION IS NULL

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
55 rows affected.


[]

In [ ]:
%%sql

SELECT * 
FROM Sales

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NV,LIEU,QTE,NB,REGION
1,BORDEAUX,33,2,NOUVELLE-AQUITAINE
1,PARIS,1,44,ÎLE-DE-FRANCE
1,RENNES,1,45,BRETAGNE
2,LYON,2,48,AUVERGNE-RHÔNE-ALPES
3,NICE,6,7,PROVENCE-ALPES-CÔTE D'AZUR
3,MERCUREY,5,49,BOURGOGNE-FRANCHE-COMTÉ
4,TOULOUSE,12,8,OCCITANIE
4,PARIS,6,44,ÎLE-DE-FRANCE
4,SENS,4,50,BOURGOGNE-FRANCHE-COMTÉ
5,MERCUREY,50,2,BOURGOGNE-FRANCHE-COMTÉ


In [ ]:
%%sql DROP TABLE IF EXISTS Client;

-- Creating Client table
CREATE TABLE Client AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2
WHERE NB IS NOT NULL
ORDER BY NB;

SELECT *
FROM Client;

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.
Done.
Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [ ]:
%%sql
SELECT DISTINCT TYPE 
FROM Client

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [ ]:
%%sql
SELECT CRU, REGION 
FROM Production
WHERE CRU = 'Pommard' OR CRU = 'Brouilly'

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [ ]:
%%sql
SELECT REGION 
FROM Production
WHERE Production.NV = (SELECT NV
        FROM WineProd
        WHERE CRU = 'Pommard'
        OR CRU = 'Brouilly')

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [ ]:
%%sql
SELECT WineSales.CRU, WineSales.MILL, QTE AS Number_of_wines_bought
FROM Sales, WineSales
WHERE Sales.NV = WineSales.NV
GROUP BY WineSales.CRU, WineSales.MILL
ORDER BY WineSales.CRU

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


CRU,MILL,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,4
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,10
Chapelle Chambertin,1973,9
Chateau Corton Grancey,1980,4
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [ ]:
%%sql
SELECT NV, COUNT(*) AS Number_of_producers
FROM Production
GROUP BY NP
HAVING COUNT(NP) > 3

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NV,Number_of_producers
54,4
99,5
87,6
57,4
60,5
72,4
87,5
77,4
45,4
81,4


Done.


NV,Number_of_producers
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [ ]:
%%sql
SELECT Producer.NP, Producer.NOM, Producer.PRENOM
FROM Producer
LEFT JOIN Production on Production.NP = Producer.NP
WHERE Production.NP IS NULL

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [ ]:
%%sql
SELECT DISTINCT Client.NB, NOM, PRENOM 
FROM Client, Sales, WineSales
WHERE Client.NB = Sales.NB
      AND
      Sales.NV = WineSales.NV
      AND
      MILL = 1980
      AND
      QTE > 0
ORDER BY Client.NB

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [ ]:
%%sql
SELECT DISTINCT Client.NB, NOM, PRENOM 
FROM Client, Sales, WineSales
WHERE Client.NB = Sales.NB
      AND
      Sales.NV = WineSales.NV
      AND
      MILL <> 1980
      AND
      QTE > 0
ORDER BY Client.NB

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
2,Artaud,Antonin
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
8,Aragon,Louis
9,Ajar,Emile
10,Andersen,Yann


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [ ]:
%%sql
SELECT DISTINCT Client.NB, NOM, PRENOM
FROM Client, Sales, WineSales
WHERE Client.NB = Sales.NB
      AND
      Sales.NV = WineSales.NV
GROUP BY Client.NB
HAVING COUNT(Client.NB) = 1 AND MILL = 1980
ORDER BY Client.NB

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NB,NOM,PRENOM
45,Giono,Jean
50,Lautreamont,None


In [ ]:
%%sql
SELECT DISTINCT Client.NB, NOM, PRENOM
FROM Client, Sales, WineSales
WHERE Client.NB = Sales.NB
      AND
      Sales.NV = WineSales.NV
GROUP BY Client.NB
HAVING COUNT(Client.NB) > 0 AND MILL = 1980
ORDER BY Client.NB

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [ ]:
%%sql
SELECT * from WineProd
WHERE MILL = 1980

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [ ]:
%%sql
SELECT * 
FROM WineSales
WHERE MILL = 1980
      AND
      (SELECT NB
        FROM Sales
        WHERE Sales.NV = WineSales.NV
        AND
        Sales.NB = 2)

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [ ]:
%%sql DROP TABLE IF EXISTS AllWines;

-- Creating All Wines table
CREATE TABLE AllWines AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE MILL = 1980;

SELECT *
FROM AllWines

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


In [ ]:
%%sql

SELECT Sales.NB, NOM, PRENOM 
FROM Sales, AllWines, Client
WHERE Sales.NV = AllWines.NV
      AND 
      Sales.NB = Client.NB
GROUP BY Sales.NB
HAVING COUNT(Sales.NB) = (SELECT COUNT(*)
                    FROM AllWines)

 * sqlite:////content/drive/MyDrive/SD202-Databases/wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
